In [4]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Directory containing folders for each class (dog, cat, tiger, etc.)
base_dir = 'dataset/'

# Image dimensions
img_width, img_height = 224, 224

# Create an image generator for training data
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2  # 20% validation data
)

# Create a generator to load images from directories
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    subset='validation'
)

# Build a model using transfer learning with MobileNetV2
def build_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
    base_model.trainable = False  # Freeze the base model layers

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        Dense(len(train_generator.class_indices), activation='softmax')  # Number of classes
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train the model
model = build_model()
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32
)

# Save the trained model
model.save('image_classifier_model.h5')

# Function to predict the image
def predict_image(image_path):
    # Load the image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, (img_width, img_height))  # Resize image to match the model input

    # Preprocess the image
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Predict the class of the image
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)[0]

    # Get the class name from the index
    class_names = list(train_generator.class_indices.keys())
    predicted_label = class_names[predicted_class]
    print(f"The image is classified as: {predicted_label}")

# Test with an image
test_image_path = 'engine.png'  # Path to the image you want to test
predict_image(test_image_path)


Found 628 images belonging to 2 classes.
Found 157 images belonging to 2 classes.
Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 14s 534ms/step - accuracy: 0.7534 - loss: 0.5122 - val_accuracy: 0.9688 - val_loss: 0.1377
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.8438 - loss: 0.5419 - val_accuracy: 0.9375 - val_loss: 0.2055
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 427ms/step - accuracy: 0.9369 - loss: 0.1705 - val_accuracy: 0.9531 - val_loss: 0.1576
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.8750 - loss: 0.2549 - val_accuracy: 0.9688 - val_loss: 0.1544
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 439ms/step - accuracy: 0.9704 - loss: 0.0854 - val_accuracy: 0.9531 - val_loss: 0.1088
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.9688 - loss: 0.0527 - val_accuracy: 0.9531 - val_loss: 0.1786
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 447ms/step - accuracy: 0.9913 - loss: 0.0318 - val_accuracy: 0.9609 - val_loss: 0.1139
Epoch 8/10
19/19 ━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 991ms/step
The image is classified as: cat


In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D